In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
os.chdir('../')
print(os.getcwd())

/home/florian/Documents/GitHub/Historical-Food-Shocks


In [ ]:
# Visualize the fit of the the 